# Clustering Text Embeddings

This notebook covers clustering techniques for text embeddings.

## Topics:
1. K-Means clustering
2. Choosing the number of clusters (Elbow, Silhouette)
3. Hierarchical clustering
4. DBSCAN for density-based clustering
5. Visualizing clusters with t-SNE/UMAP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

model = SentenceTransformer('all-MiniLM-L6-v2')

## Sample Data

Create a corpus with distinct topics for clustering.

In [ ]:
# Documents from different categories
documents = [
    # Software Installation (cluster 0)
    "How to install Python on Windows",
    "Python installation guide for beginners",
    "Setting up Python development environment",
    "Installing Python packages with pip",
    "Python virtual environment setup",
    
    # Network Issues (cluster 1)
    "WiFi connection keeps dropping",
    "Cannot connect to the internet",
    "Network troubleshooting steps",
    "Slow internet connection problems",
    "VPN connection issues",
    
    # Hardware (cluster 2)
    "Laptop screen is flickering",
    "Computer won't turn on",
    "Keyboard keys not working",
    "Mouse cursor freezing",
    "Laptop battery draining fast",
    
    # Email (cluster 3)
    "Cannot send emails from Outlook",
    "Email attachment size limit",
    "How to set up email signature",
    "Emails going to spam folder",
    "Email synchronization problems",
]

# True labels for evaluation
true_labels = [0]*5 + [1]*5 + [2]*5 + [3]*5

# Create embeddings
embeddings = model.encode(documents)
print(f"Embeddings shape: {embeddings.shape}")

## 1. K-Means Clustering

K-Means partitions data into K clusters by minimizing within-cluster variance.

In [ ]:
# Basic K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = kmeans.fit_predict(embeddings)

print("K-Means Clustering Results:")
for cluster_id in range(4):
    print(f"\nCluster {cluster_id}:")
    for i, doc in enumerate(documents):
        if labels[i] == cluster_id:
            print(f"  - {doc}")

In [ ]:
# Cluster centroids - can be used to find representative documents
centroids = kmeans.cluster_centers_
print(f"Centroid shape: {centroids.shape}")

# Find document closest to each centroid
print("\nRepresentative documents (closest to centroid):")
for cluster_id in range(4):
    # Get documents in this cluster
    cluster_mask = labels == cluster_id
    cluster_embeddings = embeddings[cluster_mask]
    cluster_docs = [doc for i, doc in enumerate(documents) if labels[i] == cluster_id]
    
    # Find closest to centroid
    distances = np.linalg.norm(cluster_embeddings - centroids[cluster_id], axis=1)
    closest_idx = np.argmin(distances)
    print(f"Cluster {cluster_id}: {cluster_docs[closest_idx]}")

## 2. Choosing the Number of Clusters

### Elbow Method
Plot inertia (within-cluster sum of squares) vs number of clusters.

In [ ]:
# Elbow method
inertias = []
K_range = range(2, 10)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(embeddings)
    inertias.append(km.inertia_)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.axvline(x=4, color='r', linestyle='--', label='Elbow at K=4')
plt.legend()

### Silhouette Score
Measures how similar an object is to its own cluster vs other clusters. Range: [-1, 1], higher is better.

In [ ]:
# Silhouette scores
silhouette_scores = []

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(embeddings)
    score = silhouette_score(embeddings, labels)
    silhouette_scores.append(score)
    print(f"K={k}: Silhouette Score = {score:.4f}")

plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, 'go-')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.tight_layout()
plt.show()

best_k = list(K_range)[np.argmax(silhouette_scores)]
print(f"\nBest K by Silhouette: {best_k}")

## 3. Hierarchical Clustering

Builds a hierarchy of clusters. Good for understanding relationships between clusters.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

# Create linkage matrix
linkage_matrix = linkage(embeddings, method='ward')

# Plot dendrogram
plt.figure(figsize=(12, 6))
dendrogram(
    linkage_matrix,
    labels=[doc[:25] + '...' for doc in documents],
    leaf_rotation=90,
    leaf_font_size=8
)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Documents')
plt.ylabel('Distance')
plt.tight_layout()
plt.show()

In [ ]:
# Agglomerative clustering with sklearn
agg = AgglomerativeClustering(n_clusters=4, linkage='ward')
agg_labels = agg.fit_predict(embeddings)

print("Agglomerative Clustering Results:")
for cluster_id in range(4):
    print(f"\nCluster {cluster_id}:")
    for i, doc in enumerate(documents):
        if agg_labels[i] == cluster_id:
            print(f"  - {doc}")

## 4. DBSCAN - Density-Based Clustering

Finds clusters of arbitrary shape. Good for detecting outliers.

In [ ]:
# DBSCAN
# eps: maximum distance between two samples in same neighborhood
# min_samples: minimum points to form a dense region

dbscan = DBSCAN(eps=0.8, min_samples=2, metric='cosine')
db_labels = dbscan.fit_predict(embeddings)

n_clusters = len(set(db_labels)) - (1 if -1 in db_labels else 0)
n_noise = list(db_labels).count(-1)

print(f"DBSCAN found {n_clusters} clusters and {n_noise} noise points")

for cluster_id in sorted(set(db_labels)):
    label = "Noise" if cluster_id == -1 else f"Cluster {cluster_id}"
    print(f"\n{label}:")
    for i, doc in enumerate(documents):
        if db_labels[i] == cluster_id:
            print(f"  - {doc}")

In [ ]:
# Finding optimal eps using k-distance plot
from sklearn.neighbors import NearestNeighbors

# Calculate distances to k nearest neighbors
k = 3
nn = NearestNeighbors(n_neighbors=k, metric='cosine')
nn.fit(embeddings)
distances, _ = nn.kneighbors(embeddings)

# Sort distances to k-th neighbor
k_distances = np.sort(distances[:, k-1])

plt.figure(figsize=(8, 4))
plt.plot(k_distances)
plt.xlabel('Points sorted by distance')
plt.ylabel(f'{k}-NN distance')
plt.title('K-Distance Plot (for choosing eps)')
plt.axhline(y=0.5, color='r', linestyle='--', label='Potential eps')
plt.legend()
plt.show()

## 5. Visualizing Clusters with t-SNE

In [ ]:
# Reduce to 2D with t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=5)
embeddings_2d = tsne.fit_transform(embeddings)

# Plot with K-Means labels
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
km_labels = kmeans.fit_predict(embeddings)

plt.figure(figsize=(12, 5))

# K-Means clusters
plt.subplot(1, 2, 1)
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                      c=km_labels, cmap='viridis', s=100)
plt.colorbar(scatter)
plt.title('K-Means Clusters (t-SNE)')

# Add labels
for i, doc in enumerate(documents):
    plt.annotate(doc[:15] + '...', 
                 (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                 fontsize=6, alpha=0.7)

# True labels
plt.subplot(1, 2, 2)
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                      c=true_labels, cmap='viridis', s=100)
plt.colorbar(scatter)
plt.title('True Labels (t-SNE)')

plt.tight_layout()
plt.show()

## 6. Cluster Quality Metrics

In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Compare predicted vs true labels
km_labels = KMeans(n_clusters=4, random_state=42, n_init=10).fit_predict(embeddings)

# Adjusted Rand Index: measures similarity between clusterings
# Range: [-1, 1], 1 = perfect match, 0 = random
ari = adjusted_rand_score(true_labels, km_labels)

# Normalized Mutual Information: measures shared information
# Range: [0, 1], 1 = perfect match
nmi = normalized_mutual_info_score(true_labels, km_labels)

# Silhouette (doesn't need true labels)
sil = silhouette_score(embeddings, km_labels)

print("Cluster Quality Metrics:")
print(f"  Adjusted Rand Index: {ari:.4f}")
print(f"  Normalized Mutual Info: {nmi:.4f}")
print(f"  Silhouette Score: {sil:.4f}")

## 7. Practical Tips

In [ ]:
# Tip 1: Normalize embeddings before clustering
embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Tip 2: Use cosine distance for K-Means (via normalization)
# After normalization, Euclidean distance ∝ cosine distance
km_norm = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_norm = km_norm.fit_predict(embeddings_norm)

print("Silhouette with normalized embeddings:", 
      silhouette_score(embeddings_norm, labels_norm))

In [ ]:
# Tip 3: Get cluster labels for new documents
def assign_cluster(text, kmeans_model, model):
    """Assign a new document to existing clusters."""
    embedding = model.encode([text])
    embedding_norm = embedding / np.linalg.norm(embedding)
    return kmeans_model.predict(embedding_norm)[0]

new_doc = "How to upgrade Python version"
cluster = assign_cluster(new_doc, km_norm, model)
print(f"'{new_doc}' -> Cluster {cluster}")

new_doc = "Printer not printing"
cluster = assign_cluster(new_doc, km_norm, model)
print(f"'{new_doc}' -> Cluster {cluster}")

## Summary

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| K-Means | Fast, scalable | Need to choose K, spherical clusters | Large datasets |
| Hierarchical | Visual dendrograms, no K needed | Slow for large data | Understanding structure |
| DBSCAN | Finds outliers, arbitrary shapes | Sensitive to eps/min_samples | Noisy data |

**Key steps:**
1. Normalize embeddings for cosine-based clustering
2. Use Elbow + Silhouette to choose K
3. Visualize with t-SNE to validate
4. Use Silhouette/ARI/NMI to evaluate quality

### Next:
Try the clustering task in `../tasks/` folder!